In [2]:
### Tools used for clean up
import numpy as np
import pandas as pd
import os

In [ ]:
def cutter(df, time_start, time_end):

    s_index = df.index[df["TIMESTAMP"]==time_start].tolist()[-1]
    e_index = df.index[df["TIMESTAMP"]==time_end].tolist()[0]
    df = df.truncate(before=s_index, after= e_index)
    df=df.reset_index(drop = True)
    df = df.set_index("TIMESTAMP")
    return df

In [ ]:
def cooked_data(path, sonic):
    df = pd.read_csv(path+sonic, na_values='NAN',header=None)
    df.fillna(value=np.nan, inplace=True)



    time, date,data = [], [], [] 
    for i in range(len(df)):
        time.append(df[0][i][-10:].replace(" ",":").replace("::",""))
        #date
        date.append(df[0][i][-21:-11].replace(" ","-"))
        #data seperated with comma's
        data.append(df[0][i][10:-25].replace("   ",",").replace(" ",""))
    time_stamp = []
    for j in range(len(df)):
        time_stamp.append(pd.Timestamp(str(date[j])+" "+time[j]))
    
    df_cook = pd.DataFrame()
    df_cook["TIMESTAMP"] = time_stamp
    df_cook['TIMESTAMP'] = pd.to_datetime(df_cook['TIMESTAMP'])
    columns=['date']
    U,V,W,T = [],[],[],[]
    for d in data:
        U.append(-1*float(d.split(",")[0]))
        V.append(-1*float(d.split(",")[1]))
        W.append(float(d.split(",")[2]))
        T.append(float(d.split(",")[3]))

    df_cook["U"] = U
    df_cook["V"] = V
    df_cook["W"] = W
    df_cook["T"] = T
    
    
    return df_cook

In [ ]:
path='http://35.12.130.8/study/charney/burn3_00_plt_syn/'
sonc_headers = ["U", "V", "W", "T" ]

In [ ]:
path_1="/home/joey/Desktop/Forestry/File_Clean_Up/10X10_Truss_Burns_May_2018/Burn-1/sonics/"


In [ ]:
burn = "Burn-1.txt"
ext = "_UVWT_"
sonic_list=["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4", "C1","C2","C3","C4","D1","D2","D3","D4"]
ckd_list=sonic_list.copy()
cleaned_lst = sonic_list.copy()
for i in range(len(sonic_list)):
    ckd_list[i]= sonic_list[i] + "UVWTfft1.txt"
    cleaned_lst[i] = sonic_list[i] + ext + burn

In [ ]:
#a1_ck_df = cooked_data(path, ckd_list[12])
a1_cl_df = cleaned_data(path_1, cleaned_lst[12])
a1_cl_df

In [ ]:
t_s, t_e = a1_cl_df["TIMESTAMP"][0], a1_cl_df["TIMESTAMP"][len(a1_cl_df)-1]

In [ ]:
a1_ck_df = cutter(cooked_data(path, ckd_list[12]),t_s,t_e)
a1_cl_df = cutter(a1_cl_df,t_s,t_e)

In [ ]:
true= a1_ck_df==a1_cl_df
indx = []
for i in range(len(true)):
    if true["U"][i] ==False or true["V"][i] == False or true['W'][i]== False or true['T'][i]==False:
        #print(true.ix[[i]])
        print(true.ix[[i]],a1_ck_df.ix[[i]],a1_cl_df.ix[[i]])
        indx.append(i)
print(indx)
print(len(indx))

In [ ]:
### Remove the bad points

a1_cl_df=a1_cl_df.drop(a1_cl_df.index[indx])
a1_ck_df=a1_ck_df.drop(a1_ck_df.index[indx])

In [ ]:
true= a1_ck_df==a1_cl_df
index = []
for i in range(len(true)):
    if true["U"][i] ==False or true["V"][i] == False or true['W'][i]== False or true['T'][i]==False:
        #print(true.ix[[i]])
        print(true.ix[[i]],a1_ck_df.ix[[i]],a1_cl_df.ix[[i]])
    

In [ ]:

print(len(a1_ck_df)/10/60)
a1_ck_df.describe()

In [ ]:
a1_cl_df.describe()

In [ ]:
def cleaned_data(path, sonic):
    df  = pd.read_csv(path+sonic, na_values='NAN',sep="\t")

    hr = list(df["Hr"])
    mn = list(df["Min"])
    sc = list(df["Sec"])
    yr = list(df["YYYY"])
    mt = list(df["MM"])
    dy = list(df["DD"])

    time = []
    for i in range(len(hr)):
        time.append(pd.Timestamp(str(yr[i])+"-"+str(mt[i])+"-"+str(dy[i])+' '+str(hr[i])+":"+str(mn[i])+":"+str(sc[i])))

    df_cleaned = pd.DataFrame()
    df_cleaned["TIMESTAMP"] = time
    df_cleaned["U"] = df["U"] 
    df_cleaned["V"] = df["U"]
    df_cleaned["W"] = df["U"]
    df_cleaned["T"] = df["T"]
    return df_cleaned, df

In [ ]:
cl_df,df = cleaned_data(path_1, "WGNover_UVWT_Burn-1.txt")

In [ ]:
indx = []
m_speed = 50
min_T = -40

for i in range(len(cl_df)):
    if np.abs(cl_df["U"][i]) > m_speed or np.abs(cl_df["V"][i])> m_speed or \
            np.abs(cl_df['W'][i])> m_speed or cl_df['T'][i]< min_T:
        print("--"*30)
        print(cl_df.iloc[[i]])
        indx.append(i)
if len(indx) ==0:
    print("Data fits these limits")
else:
    remove = input("Would you like to remove this data? (y/n):")
if remove.lower() == "y":
    df = df.drop(df.index[indx])
return df

In [ ]:
### Remove the bad points
cl_df=cl_df.drop(cl_df.index[indx])
df = df.drop(df.index[indx])

In [ ]:
cl_df["U"] *=-1

In [ ]:
cl_df.describe()

In [ ]:
df_tc  = pd.read_csv('/home/joey/Desktop/Forestry/File_Clean_Up/10X10_Truss_Burns_May_2018/Burn-1/thermal_couples/B3_thermal_couple_Burn-1.txt', na_values='NAN',sep="\t")

In [ ]:
df_tc.describe()

In [ ]:
temp=[]
for i in range(6):
    temp.append(list(df_tc.iloc[:,i]))

In [30]:
path = "/home/joey/Desktop/Forestry/File_Clean_Up/10X10_Truss_Burns_May_2018/Burn-2/sonics/"
sonic = "B1_UVWT_Burn-2.txt" 
df  = pd.read_csv(path+sonic, na_values='NAN',sep="\t")

#wind speed correction
m_speed,min_T = 50, -40
u_fctr, v_fctr = -1, -1

fmt = "Default Corrections: {}*U, {}*V, Max Wind Speed=|{}| m/s, Min Temperature = {} C  " 
print(fmt.format(u_fctr,v_fctr,m_speed,min_T))

nw_corct = input("Would you like to change these corrections? (y/n): ")
if nw_corct.lower() == "y":
    u_fctr = float(input("What to multiply the U values by?:"))
    v_fctr = float(input("What to multiply the V values by?:"))
    m_speed = float(input("What bounds do you want for the wind speed? (m/s):"))
    min_T = float(input("What is the minimum temperatue? (C):"))

fill_nan = np.nan
change_nan = input("Would you like to change the NaN's to a different value? (y/n):")
if change_nan == "y":
    fill_nan = input("What to replace NaN's with? ex: 9999:")

##For loop for all the sonics
df["U"] *= u_fctr
df["V"] *= v_fctr

indx = []
for i in range(len(df)):
    if np.abs(df["U"][i]) > m_speed or np.abs(df["V"][i])> m_speed or \
            np.abs(df['W'][i])> m_speed or df['T'][i]< min_T:
        print("--"*15,"LINE:",i,"--"*15)
        print(df.iloc[[i]])
        indx.append(i)
if len(indx) ==0:
    print("Data fits these limits")
if len(indx)!= 0:
    remove = input("Would you like to remove these lines? (y/n):")
    if remove.lower() == "y":
        df = df.drop(df.index[indx])

df.fillna(value=fill_nan, inplace=True)
return df


Default Corrections: -1.5*U, -1*V, Max Wind Speed=|50| m/s, Min Temperature = -40 C  
Would you like to change these corrections? (y/n): y
What to multiply the U values by?:-1
What to multiply the V values by?:-1
What bounds do you want for the wind speed? (m/s):50
What is the minimum temperatue? (C):-40
Would you like to change the NaN's to a different value? (y/n):n
------------------------------ LINE: 19184 ------------------------------
            U     V      W      T  YYYY  MM  DD  Hr  Min   Sec
19184 -134.15 -0.75 -62.65  16.95  2018   3   6  11   36  25.2
------------------------------ LINE: 19185 ------------------------------
           U       V      W       T  YYYY  MM  DD  Hr  Min   Sec
19185 -65.63  112.31  61.62  242.96  2018   3   6  11   36  25.3
Would you like to remove these lines? (y/n):y


In [29]:
type(u_fctr)

float

In [4]:
indx = []
for i in range(len(df)):
    if np.abs(df["U"][i]) > m_speed or np.abs(df["V"][i])> m_speed or \
            np.abs(df['W'][i])> m_speed or df['T'][i]< min_T:
        print("--"*30)
        print(df.iloc[[i]])
        indx.append(i)
if len(indx) ==0:
    print("Data fits these limits")
if len(indx)!= 0:
    remove = input("Would you like to remove this data? (y/n):")
    if remove.lower() == "y":
        df = df.drop(df.index[indx])

KeyError: 19184